# Leap seconds
* https://en.wikipedia.org/wiki/Leap_second
* https://www.americanscientist.org/article/the-future-of-time-utc-and-the-leap-second
* http://cr.yp.to/proto/utctai.html
* https://github.com/python/cpython/issues/67762 datetime: support leap seconds
* https://stackoverflow.com/questions/39686553/what-does-python-return-on-the-leap-second

## Testing difference between `datetime` and UNIX time

In [1]:
import datetime
import calendar

from typing import Iterable

In [2]:
# Check epoch start
assert (
        datetime.datetime(1970, 1, 1, tzinfo=datetime.timezone.utc)
        == datetime.datetime.fromtimestamp(0, tz=datetime.timezone.utc))

# There should be 37 leap seconds in this time period
start = datetime.datetime(1970, 1, 1, tzinfo=datetime.timezone.utc)
end = datetime.datetime(2022, 1, 1, tzinfo=datetime.timezone.utc)

datetime_seconds = (end - start).total_seconds()

timestamp_seconds = end.timestamp() - start.timestamp()

No difference

In [3]:
datetime_seconds - timestamp_seconds

0.0

seconds without leap seconds: `60 * 60 * 24 * number_of_days`

In [4]:
def is_leap_year(year):
    """Determine whether a year is a leap year."""

    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)


def days_in_years(years: Iterable[int]) -> int:
    days = 0
    for year in years:
        days += 365 + (1 if calendar.isleap(year) else 0)
    return days

SECONDS_IN_DAY = 60 * 60 * 24


seconds_without_leap = days_in_years(range(start.year, end.year)) * SECONDS_IN_DAY

In [5]:
print(
        f'{datetime_seconds =     }\n'
        f'{timestamp_seconds =    }\n'
        f'{seconds_without_leap = }'
        )

datetime_seconds =     1640995200.0
timestamp_seconds =    1640995200.0
seconds_without_leap = 1640995200


## How to get leap seconds

In [33]:
# https://stackoverflow.com/questions/65551924/how-many-seconds-elapsed-since-01-01-1970-leap-seconds-included

import os
import time
import datetime

tai_epoch = datetime.datetime(1970, 1, 1, 0, 0, 10)   # 10 seconds shift in 1972

timezone = time.tzname[0]
os.environ['TZ'] = 'right/UTC'    # special time zone without leap seconds
time.tzset()

now = datetime.datetime.utcnow()
leap_seconds = int(now.timestamp()) -  int((now - tai_epoch).total_seconds())

os.environ['TZ'] = timezone
time.tzset()

In [34]:
leap_seconds

37

In [35]:
os.environ['TZ'] = 'right/UTC'
time.tzset()
now = datetime.datetime.utcnow()
right_time = int(now.timestamp()) + leap_seconds
os.environ['TZ'] = timezone
time.tzset()

utc_time = int(time.time())
print(right_time - utc_time)

37


mktime leap second demonstration
https://github.com/python/cpython/issues/67762#issuecomment-1093676668

In [36]:
timezone = time.tzname[0]
os.environ['TZ'] = 'right/UTC'    # special time zone without leap seconds
time.tzset()

print(time.mktime((2012, 6, 30, 23, 59, 59, -1, -1, -1)))
print(time.mktime((2012, 6, 30, 23, 59, 60, -1, -1, -1)))
print(time.mktime((2012, 7, 1, 0, 0, 0, -1, -1, -1)))

os.environ['TZ'] = timezone
time.tzset()

1341100823.0
1341100824.0
1341100825.0
